In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(r'NewCleanData.csv');

In [3]:
trials = range(1,41)
bpm = [80,100,120]
gestureSize = ["S","M","L"]

In [4]:
data.DistanceCoveredSoFar = data.DistanceCoveredSoFar.astype(float)
data.VelocityMagnitude = data.VelocityMagnitude.astype(float)
data.AngleToBP1 = data.AngleToBP1.astype(float)
data.BPM = data.BPM.astype(int)
data.Position_Y = data.Position_Y.astype(float)

In [8]:
newOutputTable = pd.DataFrame({'Trial':[],'BPM':[],'GestureSize':[],'MedianVelocityYRegionOne':[],'MedianVelocityYRegionTwo':[]})

In [9]:
for g in gestureSize:
    for b in bpm:
        for i in trials: 
            sliceByTrial = data.loc[data.Trial == i] 
            sliceByGesture = sliceByTrial.loc[sliceByTrial.GestureSize == g] 
            sliceByBPM = sliceByGesture.loc[sliceByGesture.BPM == b]
            if(sliceByBPM.Trial.size == 0):
               continue
            
            # =======================================
            # ============ REGION 1 =================
            # =======================================
            
            globalHighestPointIndex = sliceByBPM.Position_Y.idxmax()
            localHighestPointIndex = globalHighestPointIndex - sliceByBPM.index[0]
            
            sliceToGetPositiveYVelocity = sliceByBPM.loc[sliceByBPM.Velocity_Y > 0]
            globalIndexFirstPositiveYVelocity = sliceToGetPositiveYVelocity.index[0]
            
            localIndexFirstPositiveYVelocity = globalIndexFirstPositiveYVelocity - sliceByBPM.index[0]
            
            RegionOne = sliceByBPM[localIndexFirstPositiveYVelocity:localHighestPointIndex+1]
            
            
            # =======================================
            # ============ REGION 2 =================
            # =======================================
            
            smallestYPoint_RegionOneIndex = RegionOne.Position_Y.idxmin()
            
            y_positionHighestPoint = sliceByBPM.Position_Y[globalHighestPointIndex]
            y_positionLowestPoint  = sliceByBPM.Position_Y[smallestYPoint_RegionOneIndex]
            
            y_distance = np.absolute(y_positionHighestPoint - y_positionLowestPoint)
            y_distanceWithinRegion2 = y_distance/2
            
            portionAfterRegionOne = sliceByBPM[localHighestPointIndex+1:]
            
            RegionTwo = portionAfterRegionOne[np.absolute(portionAfterRegionOne.Position_Y - y_positionHighestPoint) < y_distanceWithinRegion2]
            
            # =======================================
            # ============== Median =================
            # =======================================
            
            currentMedianVelocityYRegionTwo = np.median(RegionTwo.Velocity_Y)
            currentMedianVelocityYRegionOne = np.median(RegionOne.Velocity_Y)
            
            newOutputTable = newOutputTable.append({'Trial': i,'BPM':b,'GestureSize':g,'MedianVelocityYRegionOne':currentMedianVelocityYRegionOne,'MedianVelocityYRegionTwo':currentMedianVelocityYRegionTwo}, ignore_index = True)
            

In [10]:
newOutputTable

,Trial,BPM,GestureSize,MedianVelocityYRegionOne,MedianVelocityYRegionTwo
0,1.0,80.0,S,0.990901,-0.525427
1,2.0,80.0,S,0.859788,-0.488448
2,3.0,80.0,S,0.934681,-0.586229
3,4.0,80.0,S,1.030644,-0.454952
4,5.0,80.0,S,1.078234,-0.465989
5,6.0,80.0,S,0.962615,-0.594246
6,7.0,80.0,S,0.991963,-0.630125
7,8.0,80.0,S,1.036439,-0.627624
8,9.0,80.0,S,1.143092,-0.638987
9,10.0,80.0,S,1.152874,-0.527127


In [11]:
newOutputTable.to_csv(r'MedianVelocityYPerTrial.csv')